In [1]:
# coding: utf-8

import numpy as np
import pandas as pd
import pickle
import gzip
from collections import Counter
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine

In [3]:
glove_model = KeyedVectors.load_word2vec_format("glove.6B/glove.6B.300d.vocab.w2v.bin")
#cbow_model = KeyedVectors.load_word2vec_format("glove.6B/cbow.400d.vocab.w2v.bin")

In [8]:
cbow_model = KeyedVectors.load_word2vec_format("glove.6B/cbow.400d.vocab.w2v.bin")

In [9]:
cbow_model.most_similar('cow')

[('cows', 0.6693729162216187),
 ('calf', 0.506991446018219),
 ('udders', 0.4867766499519348),
 ('goat', 0.47262561321258545),
 ('pig', 0.4672166705131531),
 ('udder', 0.4562602639198303),
 ('calves', 0.45408329367637634),
 ('cattle', 0.45118770003318787),
 ('sheep', 0.44824469089508057),
 ('herd', 0.43862858414649963)]

In [3]:
glove_model.most_similar('cow')

[('cows', 0.7157833576202393),
 ('mad', 0.6735858917236328),
 ('bovine', 0.6667904853820801),
 ('cattle', 0.6248130798339844),
 ('sheep', 0.596968412399292),
 ('beef', 0.5827497243881226),
 ('goat', 0.5626482963562012),
 ('pig', 0.5351741313934326),
 ('meat', 0.5321629643440247),
 ('animal', 0.5035290122032166)]

In [10]:
glove_model.most_similar('horse')

[('horses', 0.8141624927520752),
 ('dog', 0.5369751453399658),
 ('riding', 0.5290381908416748),
 ('racing', 0.5160472989082336),
 ('rode', 0.5007773637771606),
 ('jockey', 0.48490625619888306),
 ('ride', 0.48027461767196655),
 ('cat', 0.4732130765914917),
 ('stallion', 0.47192591428756714),
 ('race', 0.46755677461624146)]

In [11]:
cbow_model.most_similar('horse')

[('horses', 0.7770479917526245),
 ('pony', 0.5609266757965088),
 ('stallion', 0.55982506275177),
 ('riderless', 0.5317178964614868),
 ('foal', 0.5293401479721069),
 ('horseman', 0.4821648597717285),
 ('hooves', 0.44404685497283936),
 ('gallop', 0.4353393316268921),
 ('mule', 0.4334191083908081),
 ('galloping', 0.4329950213432312)]

In [12]:
glove_model.similar_by_vector(glove_model['cow'] + glove_model['riding'])

[('cow', 0.7939551472663879),
 ('riding', 0.7633981108665466),
 ('cows', 0.6142357587814331),
 ('horse', 0.6075267791748047),
 ('horses', 0.5751385688781738),
 ('mad', 0.5563023090362549),
 ('cattle', 0.5457651019096375),
 ('sheep', 0.5411986708641052),
 ('rode', 0.4979754090309143),
 ('ride', 0.47951769828796387)]

In [13]:
cbow_model.similar_by_vector(cbow_model['cow'] + cbow_model['riding'])

[('riding', 0.757603108882904),
 ('cow', 0.734711766242981),
 ('horse', 0.4997519850730896),
 ('cows', 0.4960385262966156),
 ('pony', 0.49214795231819153),
 ('riderless', 0.4482153356075287),
 ('horses', 0.43953239917755127),
 ('rider', 0.4353545904159546),
 ('udders', 0.430571585893631),
 ('bull', 0.4166943430900574)]

In [14]:
cbow_model.similar_by_vector(cbow_model['cow'] + cbow_model['bear'] + cbow_model['guy'])

[('cow', 0.7070010304450989),
 ('bear', 0.6406359672546387),
 ('guy', 0.5605200529098511),
 ('bears', 0.46947741508483887),
 ('cows', 0.4568237066268921),
 ('goat', 0.43706417083740234),
 ('dog', 0.4228760004043579),
 ('binky', 0.42065703868865967),
 ('doofus', 0.3985612988471985),
 ('pig', 0.39792752265930176)]

In [18]:
glove_model.similar_by_vector(glove_model['cow'] + glove_model['bear'] + glove_model['guy'])

[('bear', 0.7004120349884033),
 ('cow', 0.6997702121734619),
 ('guy', 0.6032382249832153),
 ('dog', 0.5678098201751709),
 ('mad', 0.5576112270355225),
 ('thing', 0.5364335775375366),
 ('kind', 0.5276697874069214),
 ('big', 0.523923933506012),
 ('man', 0.5139811038970947),
 ('kid', 0.5113055109977722)]

In [19]:
glove_model.similarity('cow','horse')

0.42092965899493234

In [23]:
1-cosine(glove_model['cow'],glove_model['horse'])

0.4209296481949768

In [34]:
cow_horse_sim = Counter()

In [35]:
for word in glove_model.vocab:
    if word not in ['cow','horse','horses']:
        cow_horse_sim[word] = 1-cosine(glove_model['cow']+glove_model[word],glove_model['horse'])

In [36]:
cow_horse_sim.most_common(10)

[('racing', 0.62743886243569857),
 ('jockey', 0.61821113327669852),
 ('riding', 0.60752679765240636),
 ('rode', 0.60303559755114522),
 ('colt', 0.59602460439156923),
 ('race', 0.59558967883424219),
 ('ride', 0.58426705084976849),
 ('stallion', 0.57224129628173059),
 ('rides', 0.57220820502066805),
 ('dog', 0.57037978398090328)]

In [37]:
cow_horse_sim2 = Counter()

In [38]:
for first_word,_ in cow_horse_sim.most_common(20):
    cow_horse_sim2[(first_word,)] = cow_horse_sim[first_word]
    for second_word in glove_model.vocab:
        if second_word not in ['cow','horse','horses',first_word]:
            cow_horse_sim2[(first_word,second_word)] = 1-cosine(glove_model['cow']+glove_model[second_word]+glove_model[first_word],glove_model['horse'])

In [39]:
cow_horse_sim2.most_common(10)

[(('riding', 'colt'), 0.70302507153664229),
 (('colt', 'riding'), 0.70302507153664229),
 (('racing', 'colt'), 0.69822233277287649),
 (('colt', 'racing'), 0.69822233277287649),
 (('racing', 'rode'), 0.69394585927646757),
 (('rode', 'racing'), 0.69394585927646757),
 (('racing', 'stallion'), 0.69006248148526095),
 (('stallion', 'racing'), 0.69006248148526095),
 (('racing', 'dog'), 0.68958321550557955),
 (('dog', 'racing'), 0.68958321550557955)]

In [40]:
n = 0.5
1-cosine(((n*glove_model['cow'])+((1-n)*glove_model['racing'])),glove_model['horse'])

0.62743886243569857

In [44]:
for n in range(1,10):
    print(n/10,1-cosine((((n/10)*glove_model['cow'])+((1-n/10)*glove_model['racing'])),glove_model['horse']))
    

0.1 0.550573780375
0.2 0.583873419828
0.3 0.611623048233
0.4 0.628005486289
0.5 0.627438862436
0.6 0.607473320468
0.7 0.570607207475
0.8 0.523023664075
0.9 0.471460196007


In [45]:
glove_model.similar_by_vector(glove_model['cow'] + glove_model['dog'] + glove_model['bike'])

[('dog', 0.795136570930481),
 ('cow', 0.7213115692138672),
 ('bike', 0.6684139370918274),
 ('dogs', 0.6264644861221313),
 ('horse', 0.6207606792449951),
 ('bicycle', 0.5497134923934937),
 ('cat', 0.5461453199386597),
 ('goat', 0.5461424589157104),
 ('animal', 0.5442163944244385),
 ('sheep', 0.5433530211448669)]

In [49]:
glove_model.most_similar('horse',topn=20)

[('horses', 0.8141624927520752),
 ('dog', 0.5369751453399658),
 ('riding', 0.5290381908416748),
 ('racing', 0.5160472989082336),
 ('rode', 0.5007773637771606),
 ('jockey', 0.48490625619888306),
 ('ride', 0.48027461767196655),
 ('cat', 0.4732130765914917),
 ('stallion', 0.47192591428756714),
 ('race', 0.46755677461624146),
 ('colt', 0.46563082933425903),
 ('saddle', 0.46305644512176514),
 ('camel', 0.4590829610824585),
 ('rides', 0.45908161997795105),
 ('trainer', 0.4521074891090393),
 ('bred', 0.45107370615005493),
 ('pony', 0.4498513340950012),
 ('sheep', 0.444318562746048),
 ('bull', 0.44192397594451904),
 ('donkey', 0.43909531831741333)]

In [50]:
len(glove_model.vocab)

6696

In [51]:
glove_plus = KeyedVectors.load_word2vec_format("glove.6B/glove.6B.300d.vocabplus.w2v.bin")

In [52]:
len(glove_plus.vocab)

8579

In [53]:
glove_plus.most_similar('horse')

[('horses', 0.8141624927520752),
 ('dog', 0.5369751453399658),
 ('riding', 0.5290381908416748),
 ('racing', 0.5160472989082336),
 ('rode', 0.5007773637771606),
 ('jockey', 0.48490625619888306),
 ('ride', 0.48027461767196655),
 ('cat', 0.4732130765914917),
 ('stallion', 0.47192591428756714),
 ('race', 0.46755677461624146)]

In [64]:
glove_model.similar_by_vector(glove_model['animal'] + glove_model['bull'] + glove_model['cow'])

[('cow', 0.8207373023033142),
 ('animal', 0.771302342414856),
 ('bull', 0.7218084335327148),
 ('animals', 0.670396625995636),
 ('cows', 0.6582960486412048),
 ('cattle', 0.6202667951583862),
 ('sheep', 0.6193576455116272),
 ('mad', 0.5810044407844543),
 ('meat', 0.5745887756347656),
 ('dog', 0.5720890760421753)]

In [68]:
glove_model.similar_by_vector(glove_model['cow'] + glove_model['seat'],topn=20)

[('seat', 0.7422691583633423),
 ('cow', 0.7356062531471252),
 ('cows', 0.5139919519424438),
 ('seats', 0.5068806409835815),
 ('mad', 0.47221899032592773),
 ('cattle', 0.46913474798202515),
 ('sheep', 0.44709721207618713),
 ('bovine', 0.435454398393631),
 ('sitting', 0.41613683104515076),
 ('beef', 0.4071590304374695),
 ('horse', 0.4055815041065216),
 ('riding', 0.39647307991981506),
 ('dog', 0.3770301938056946),
 ('foot', 0.3724724352359772),
 ('animal', 0.3707016706466675),
 ('goat', 0.36898183822631836),
 ('milk', 0.36878344416618347),
 ('back', 0.3686487376689911),
 ('meat', 0.368600457906723),
 ('farm', 0.36848998069763184)]

In [76]:
sum_vec = glove_model['cow']
for w in ['person','elephant','black','man','animal']:
    print(1-cosine(sum_vec,glove_model['horse']))
    sum_vec = sum_vec + glove_model[w]
    print("+",w)
    print(1-cosine(sum_vec,glove_model['horse']))
    
print(glove_model.similar_by_vector(sum_vec))

0.420929648195
+ person
0.458124158126
0.458124158126
+ elephant
0.530731503567
0.530731503567
+ black
0.525253770401
0.525253770401
+ man
0.520237539256
0.520237539256
+ animal
0.538562516329
[('animal', 0.7068608999252319), ('man', 0.6890851259231567), ('elephant', 0.662988543510437), ('cow', 0.6541603803634644), ('animals', 0.6370037198066711), ('woman', 0.6142318248748779), ('person', 0.6114072799682617), ('dog', 0.604219913482666), ('black', 0.5613945722579956), ('boy', 0.5487518906593323)]


In [84]:
glove_model.similar_by_vector(glove_model['train'] + glove_model['car'] + glove_model['suitcase'],topn=20)

[('car', 0.7843878269195557),
 ('train', 0.7594270706176758),
 ('truck', 0.6765108108520508),
 ('bus', 0.6656845808029175),
 ('suitcase', 0.6476494073867798),
 ('cars', 0.631690263748169),
 ('passenger', 0.6314556002616882),
 ('vehicle', 0.6233620643615723),
 ('trains', 0.5941781997680664),
 ('luggage', 0.580843985080719),
 ('parked', 0.5775442123413086),
 ('driver', 0.5527359843254089),
 ('carrying', 0.5359137058258057),
 ('taxi', 0.5304840803146362),
 ('passengers', 0.5259625911712646),
 ('vehicles', 0.5113798379898071),
 ('plane', 0.5052343606948853),
 ('wagon', 0.5049293041229248),
 ('jeep', 0.5048753619194031),
 ('buses', 0.50296950340271)]

In [85]:
glove_model.similar_by_vector(glove_model['train'])

[('train', 1.0000001192092896),
 ('trains', 0.8230966329574585),
 ('bus', 0.670767068862915),
 ('rail', 0.635927140712738),
 ('freight', 0.5893657803535461),
 ('passenger', 0.5863174200057983),
 ('buses', 0.5471792817115784),
 ('subway', 0.5458871126174927),
 ('passengers', 0.5390786528587341),
 ('station', 0.5274492502212524)]

In [87]:
glove_model['train'][0]

-0.037207998

In [90]:
dil_vec = (glove_model['train']*glove_model['train']*glove_model['car']) + (0.4*(glove_model['train']*glove_model['car']*glove_model['car']))

In [97]:
glove_model.similar_by_vector(glove_model['train'] + 0.2*glove_model['car'] + 0.2*glove_model['truck'])

[('train', 0.9641594886779785),
 ('trains', 0.7808172106742859),
 ('bus', 0.7379264831542969),
 ('truck', 0.6669365167617798),
 ('car', 0.6652240753173828),
 ('passenger', 0.6368261575698853),
 ('rail', 0.621604323387146),
 ('freight', 0.5962470769882202),
 ('cars', 0.5927486419677734),
 ('buses', 0.5894615650177002)]

In [100]:
glove_model.most_similar('truck')

[('trucks', 0.7520503997802734),
 ('car', 0.7350622415542603),
 ('vehicle', 0.6715237498283386),
 ('pickup', 0.6509097814559937),
 ('tractor', 0.6300186514854431),
 ('driver', 0.6279593110084534),
 ('parked', 0.6254948377609253),
 ('bus', 0.6185721755027771),
 ('vehicles', 0.5857246518135071),
 ('cars', 0.583194375038147)]

In [106]:
sum_vec = glove_model['train'] + glove_model['car'] + glove_model['truck']

In [107]:
1-cosine(sum_vec,glove_model['train'])

0.76653252791043003

In [108]:
1-cosine(sum_vec,glove_model['car'])

0.87775075029811234

In [109]:
1-cosine(sum_vec,glove_model['bus'])

0.73491121992195219

In [110]:
1-cosine(glove_model['train'],glove_model['bus'])

0.67076714049883301

In [21]:
seeds = ['car','horse','dog','cat','bed','blue','man','left']
for seed in seeds:
    for w in glove_model.vocab:
        if w != seed:
            sim_sum = glove_model.similar_by_vector(glove_model[seed] + glove_model[w],topn=2)
            sim_word = sim_sum[0][0]
            if sim_word not in [seed,w]:
                print(seed,"+",w," = ",sim_sum)
        

car + trucks  =  [('truck', 0.8649938106536865), ('trucks', 0.8598874807357788)]
car + flatbed  =  [('truck', 0.7964094877243042), ('car', 0.7792336940765381)]
horse + cows  =  [('horses', 0.8581856489181519), ('cows', 0.8287484645843506)]
horse + giraffes  =  [('horses', 0.7635481953620911), ('horse', 0.7488400936126709)]
horse + donkeys  =  [('horses', 0.8495331406593323), ('horse', 0.8340033292770386)]
horse + animals  =  [('horses', 0.8394337892532349), ('horse', 0.8361673355102539)]
man + extremely  =  [('very', 0.8110775947570801), ('man', 0.8020899295806885)]
left + fourth  =  [('third', 0.8497422933578491), ('fourth', 0.8385612964630127)]
left + sixth  =  [('third', 0.8158323168754578), ('fifth', 0.8147475719451904)]


In [22]:
seeds = ['car','horse','dog','cat','bed','blue','man','left']
for seed in seeds:
    for w in cbow_model.vocab:
        if w != seed:
            sim_sum = cbow_model.similar_by_vector(cbow_model[seed] + cbow_model[w],topn=2)
            sim_word = sim_sum[0][0]
            if sim_word not in [seed,w]:
                print(seed,"+",w," = ",sim_sum)

In [24]:
v = cbow_model['car'] + cbow_model['trucks']

In [25]:
v.shape

(400,)

In [28]:
assert(v[0] == cbow_model['car'][0] + cbow_model['trucks'][0])

In [29]:
v[0] == cbow_model['car'][0] + cbow_model['trucks'][0]

True

In [30]:
np.dot(cbow_model['car'],cbow_model['trucks'])

3.6558504

In [51]:
cbow_model['cow']

array([ -2.60758013e-01,  -7.03340024e-02,  -1.40745997e-01,
        -9.27499961e-03,   5.69950007e-02,   8.80189985e-02,
         1.69313997e-01,   1.74356997e-01,  -1.31465003e-01,
        -1.48888007e-01,  -1.43814996e-01,   1.94058999e-01,
         3.05460002e-02,  -4.10030000e-02,  -8.12400039e-03,
        -2.42277995e-01,   1.01135001e-01,  -3.65038007e-01,
        -1.62480008e-02,  -2.69403994e-01,   4.83510010e-02,
         2.40119994e-01,  -3.15379985e-02,  -1.23741999e-01,
         8.65100026e-02,   5.02219982e-02,  -9.99509990e-02,
        -1.29056007e-01,  -3.54645014e-01,   4.52449992e-02,
        -1.94554999e-01,  -3.25060003e-02,  -9.36419964e-02,
         1.12342000e-01,  -5.97280003e-02,  -1.14298001e-01,
        -7.14699998e-02,   5.72489984e-02,  -6.81520030e-02,
        -9.24640000e-02,   1.07070003e-02,   2.08528996e-01,
        -1.39103994e-01,   3.04028004e-01,   1.57637000e-01,
         1.73719991e-02,  -1.82906002e-01,  -2.78196007e-01,
        -1.65650006e-02,

In [50]:
cbow_model.similar_by_vector(cbow_model['cow'] + 0.4*cbow_model['riding'] + 0.4*cbow_model['leg'],topn=5)

[('cow', 0.8819707036018372),
 ('cows', 0.5861097574234009),
 ('calf', 0.5459157228469849),
 ('udders', 0.5264468789100647),
 ('horse', 0.4965650737285614)]

In [42]:
cbow_model.similar_by_vector(cbow_model['cow'] * cbow_model['riding'],topn=5)

[('dried', 0.21771013736724854),
 ('mayor', 0.19192665815353394),
 ('acorns', 0.1889016330242157),
 ('leaf', 0.18425947427749634),
 ('leafs', 0.17793048918247223)]

In [122]:
glove_model.similar_by_vector(glove_model['cow'],topn=10)

[('cow', 1.0),
 ('cows', 0.7157833576202393),
 ('mad', 0.6735858917236328),
 ('bovine', 0.6667905449867249),
 ('cattle', 0.6248130798339844),
 ('sheep', 0.596968412399292),
 ('beef', 0.5827497243881226),
 ('goat', 0.5626482963562012),
 ('pig', 0.5351741313934326),
 ('meat', 0.5321629643440247)]

In [31]:
glove_model.similarity('cow','riding')

0.21339471356266515

In [32]:
glove_model.similarity('cow','left')

0.12516341304137707

In [33]:
glove_model.similarity('cow','cows')

0.71578337006980597

In [34]:
glove_model.similarity('cow','horse')

0.42092965899493234

In [120]:
glove_model.similar_by_vector(glove_model['couch'],topn=10)

[('couch', 1.0),
 ('sofa', 0.6295238733291626),
 ('couches', 0.5716592669487),
 ('sitting', 0.5219179391860962),
 ('lounging', 0.49838778376579285),
 ('cushions', 0.4883556663990021),
 ('armchair', 0.48371925950050354),
 ('bed', 0.4826846718788147),
 ('recliner', 0.48102957010269165),
 ('asleep', 0.47139671444892883)]

In [125]:
glove_model.similar_by_vector(glove_model['couch']+glove_model['sofa']+glove_model['pillows'],topn=10)

[('sofa', 0.8632580041885376),
 ('couch', 0.8238006234169006),
 ('pillows', 0.7935529947280884),
 ('couches', 0.6973887085914612),
 ('cushions', 0.6853867173194885),
 ('mattress', 0.600668728351593),
 ('upholstered', 0.5891541242599487),
 ('bed', 0.5732811689376831),
 ('pillow', 0.5685983896255493),
 ('curtains', 0.5427563786506653)]

In [128]:
glove_model.similar_by_vector(glove_model['couch']+glove_model['red']+glove_model['sofa'],topn=10)

[('sofa', 0.8043059706687927),
 ('couch', 0.7828922271728516),
 ('red', 0.5984403491020203),
 ('couches', 0.5368978977203369),
 ('cushions', 0.5299590826034546),
 ('sitting', 0.5292019248008728),
 ('bed', 0.5176485776901245),
 ('room', 0.5005917549133301),
 ('velvet', 0.49743711948394775),
 ('pink', 0.4857308268547058)]

In [137]:
glove_model.similar_by_vector(glove_model['bike']+glove_model['truck']+glove_model['vehicle']+glove_model['car'],topn=10)

[('car', 0.9081412553787231),
 ('vehicle', 0.8591530323028564),
 ('truck', 0.8455815315246582),
 ('cars', 0.7415859699249268),
 ('vehicles', 0.7174458503723145),
 ('driver', 0.6897912621498108),
 ('bicycle', 0.6685762405395508),
 ('motorcycle', 0.6672142744064331),
 ('bike', 0.6663811206817627),
 ('driving', 0.655143141746521)]

In [130]:
a = np.array([0.5,0.4,0.1])
b = np.array([0.5,0.1,0.4])
c = np.array([0.5,0.3,0.2])


In [134]:
np.sum(a*np.log(a/b))

0.41588830833596713

In [135]:
np.sum(a*np.log(a/c))

0.04575811092471789

In [70]:
refdf_path = "refcoco_refdf_heads.json.gz"
refdf = pd.read_json(refdf_path,compression='gzip', orient='split')

In [71]:
refdf

,i_corpus,image_id,region_id,names,refexps,cat,ix_Xfile
0,1,581857,1719310,"[person, lady, shirt]","[the lady with the blue shirt, lady with back ...",1,34271
1,1,581857,463958,"[person, woman]","[woman in gray shirt facing camera on right, w...",1,34270
2,1,581839,495152,[person],[person standing u],1,34771
3,1,581839,485695,"[person, girl, lady, woman]","[lady sitting on right, right girl on floor, w...",1,34770
4,1,581789,453177,"[person, woman]","[woman, left person, woman under suitcase]",1,32032
5,1,581789,440706,"[person, man]","[man in jacket, man trying to help with suitca...",1,32031
6,1,581766,1097431,[toilet],"[left most toilet, far left toilet]",70,46501
7,1,581766,1093269,[toilet],[],70,46500
8,1,581739,1154905,[vase],"[right vase, right most vase]",86,47155
9,1,581739,1152539,[vase],"[middle vase, center vase]",86,47154


In [73]:
np.array(refdf.ix_Xfile)

array([34271, 34270, 34771, ..., 27600, 41159, 41158])

In [77]:
np.zeros((3,))

array([ 0.,  0.,  0.])

In [80]:
np.hstack([np.zeros((3,)), np.ones((3,))])

array([ 0.,  0.,  0.,  1.,  1.,  1.])

In [76]:
np.random.choice([2,3,6,9,4],2,replace=False)

array([3, 9])

In [4]:
refdf_path = "refcoco_refdf_heads.json.gz"

In [35]:
refdf = pd.read_json(refdf_path,compression='gzip', orient='split')

In [36]:
refdf.head()

,i_corpus,image_id,region_id,names,cat,ix_Xfile
0,1,581857,1719310,"[lady, shirt, person]",1,34271
1,1,581857,463958,"[woman, person]",1,34270
2,1,581839,495152,[person],1,34771
3,1,581839,485695,"[girl, lady, woman, person]",1,34770
4,1,581789,453177,"[woman, person]",1,32032


In [32]:
refdf[refdf.region_id == 57115]

,i_corpus,image_id,region_id,names,cat
1675,1,563447,57115,[horse],19


In [33]:
refdf[refdf.image_id == 480023]

,i_corpus,image_id,region_id,names,cat
8905,1,480023,1745965,"[person, hand]",1
8906,1,480023,1292891,"[person, shirt]",1
8907,1,480023,498741,"[person, man, guy]",1
8908,1,480023,438029,"[person, girl, woman]",1


In [34]:
refdf[refdf.region_id == 448467]

,i_corpus,image_id,region_id,names,cat
7498,1,495875,448467,"[person, man, guy]",1


In [7]:
len(refdf)

50000

In [9]:
vocab = []
for n in refdf.names:
    vocab += list(n)

In [12]:
len(list(set(vocab)))

2901

In [19]:
"" in vocab

False

In [14]:
list(set(vocab))

['clump',
 'sculpture',
 'badass',
 'elks',
 'driver',
 'flowers',
 'lavender',
 'pancakes',
 'stand',
 'rabbit',
 'chunks',
 'oclock',
 'beam',
 'ber',
 'individual',
 'chic',
 'brixton',
 'club',
 'women',
 'sammy',
 'buss',
 'necklace',
 'eating',
 'model',
 'poerson',
 'flower',
 'blender',
 'banannas',
 'batte',
 'armhand',
 'strawberries',
 'delk',
 'hoddie',
 'universal',
 'rite',
 'garaffe',
 'xxx',
 'mascot',
 'character',
 'smack',
 'carpet',
 'kernel',
 'overalls',
 'lips',
 'macbook',
 'stuffed',
 'dials',
 'uniform',
 'see',
 'hor',
 'tables',
 'cranberry',
 'menu',
 'hunk',
 'hello',
 'kity',
 'penguin',
 'truckvan',
 'tractor',
 'decorative',
 'moo',
 'popcorn',
 'trash',
 'topping',
 'stick',
 'pepsi',
 'berries',
 'armrest',
 'sink',
 'outfit',
 'whitew',
 'creamy',
 'bear',
 'wannabe',
 'rght',
 'ho',
 'sort',
 'glare',
 'sandiwch',
 'vodka',
 'priest',
 'page',
 'gril',
 'plater',
 'listerine',
 'backrest',
 'geisha',
 'mcneff',
 'lounger',
 'metor',
 'plaid',
 'ok',

In [16]:
glove_model = KeyedVectors.load_word2vec_format("glove.6B/glove.6B.300d.w2v.bin")

In [18]:
glove_model['person'];

In [20]:
vocab = list(set(vocab))


In [21]:
glove_vocab = [ w for w in vocab if w in glove_model]

In [22]:
len(glove_vocab)

2393

In [25]:
print('person',' '.join([str(i) for i in glove_model['person']]))

person -0.55598 0.027967 -0.32289 -0.075042 -0.13179 0.13271 -0.15578 0.11468 -0.055989 -1.6266 0.12912 -0.20525 -0.2162 -0.087432 -0.085335 0.033107 -0.22467 -0.099248 -0.21777 -0.25671 0.063234 0.50175 -0.16002 0.30961 0.20912 -0.29045 0.042019 -0.022472 0.24542 -0.075772 -0.56552 0.62501 -0.35209 -0.60992 -0.93654 0.25625 0.21782 -0.24591 -0.53093 0.20034 0.022471 0.073258 -0.029808 0.082042 -0.26838 0.011861 0.5919 0.008274 -0.07756 0.37252 0.12574 0.05661 0.61548 0.19826 -0.053163 0.27715 0.15825 0.13941 0.35109 -0.21844 0.10418 0.089417 0.11362 0.25903 -0.10444 -0.30457 -0.36033 -0.22062 -0.47356 0.48017 0.037563 -0.42485 -0.026248 -0.14516 0.012868 0.041353 0.1957 0.34752 0.15859 0.097584 -0.1535 0.24506 -0.12282 0.24122 -0.42262 -0.20683 -0.26847 -0.22687 -0.038363 0.53815 -0.77009 0.3295 -0.42145 -0.29034 0.59593 -0.041209 -0.3399 0.3474 0.14502 -0.21154 -0.29105 0.00061433 -0.034839 -0.06446 0.50316 0.34938 0.61775 -0.15638 -0.001032 0.13306 -0.0085934 0.5276 0.22607 -0.19192

In [26]:
from gensim.scripts.glove2word2vec import glove2word2vec


In [27]:
glove2word2vec("glove.6B/glove.6B.300d.heads.txt", "glove.6B/glove.6B.300d.heads.w2v.bin")

(779, 300)

In [52]:
from nltk.corpus import wordnet as wn


In [69]:
for s0 in wn.synsets('cake'):
    print(s0)
    print(s0.lemma_names())
    for hsyn in s0.hypernyms():
        print("hsyn",hsyn)
        print(hsyn.lemma_names())
        for hsyn2 in hsyn.hypernyms():
            print("hsyn2",hsyn2)
            print(hsyn2.lemma_names())

Synset('cake.n.01')
['cake', 'bar']
hsyn Synset('block.n.01')
['block']
hsyn2 Synset('artifact.n.01')
['artifact', 'artefact']
Synset('patty.n.01')
['patty', 'cake']
hsyn Synset('dish.n.02')
['dish']
hsyn2 Synset('nutriment.n.01')
['nutriment', 'nourishment', 'nutrition', 'sustenance', 'aliment', 'alimentation', 'victuals']
Synset('cake.n.03')
['cake']
hsyn Synset('baked_goods.n.01')
['baked_goods']
hsyn2 Synset('food.n.02')
['food', 'solid_food']
Synset('coat.v.03')
['coat', 'cake']
hsyn Synset('cover.v.02')
['cover', 'spread_over']
hsyn2 Synset('touch.v.05')
['touch', 'adjoin', 'meet', 'contact']


In [58]:
s0.instance_hypernyms()

[]

In [59]:
s0.lemma_names()

['person', 'individual', 'someone', 'somebody', 'mortal', 'soul']